## 任务5 - 模型调优（2天）
任务5：使用网格搜索法对5个模型进行调优（调参时采用五折交叉验证的方式），并进行模型评估，记得展示代码的运行结果。 时间：2天



## 任务5：使用网格搜索法对5个模型进行调优（调参时采用五折交叉验证的方式），并进行模型评估。

### 数据载入

In [1]:
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 载入数据
with open('final.pkl', 'rb') as f:
    final_data = pickle.load(f)
# data = pd.read_csv("./data.csv",encoding='gbk')
# y=data.status
# 划分训练集测试集
X, y = final_data[final_data.columns.drop("status")], final_data['status']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2018)


# 特征归一化
std = StandardScaler()
X_train = std.fit_transform(X_train)
X_test = std.transform(X_test)

D:\Anaconda2\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
D:\Anaconda2\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
D:\Anaconda2\lib\site-packages\ipykernel_launcher.py:19: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.


### 模型评估

In [2]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score,roc_curve, auc
import matplotlib.pyplot as plt
%matplotlib inline
model_score_train = []
roc_score_train = []
model_score_test = []
roc_score_test = []

In [3]:
from sklearn.metrics import accuracy_score, roc_auc_score,precision_score

def model_metrics(clf, X_train, X_test, y_train, y_test):
    # 预测
    y_train_pred = clf.predict(X_train)
    y_test_pred = clf.predict(X_test)
    
    y_train_proba = clf.predict_proba(X_train)[:,1]
    y_test_proba = clf.predict_proba(X_test)[:,1]
    
    
    # 准确率
    print(u'[准确率]')
    print(u'训练集：'+ '%.4f'%accuracy_score(y_train, y_train_pred))
    print(u'测试集：'+ '%.4f'%accuracy_score(y_test, y_test_pred))
    
    # 精准率
    print(u'[精准率]')
    print(u'训练集：'+ '%.4f'%precision_score(y_train, y_train_pred))
    print(u'测试集：'+ '%.4f'%precision_score(y_test, y_test_pred))

    # 召回率
    print(u'[召回率]')
    print(u'训练集：'+ '%.4f'%recall_score(y_train, y_train_pred))
    print(u'测试集：'+ '%.4f'%recall_score(y_test, y_test_pred))
    
    # f1-score
    print(u'[f1-score]')
    print(u'训练集：'+ '%.4f'%f1_score(y_train, y_train_pred))
    print(u'测试集：'+ '%.4f'%f1_score(y_test, y_test_pred))
    
    # auc取值：用roc_auc_score或auc
    print(u'[auc值]')
    print(u'训练集：'+ '%.4f'%roc_auc_score(y_train, y_train_proba))
    print(u'测试集：'+ '%.4f'%roc_auc_score(y_test, y_test_proba))

In [4]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from xgboost.sklearn import XGBClassifier
from lightgbm.sklearn import LGBMClassifier

### LR模型

In [5]:
lr = LogisticRegression()
param = {'C': [1e-3,0.01,0.1,1,10,100,1e3], 'penalty':['l1', 'l2']}

gsearch = GridSearchCV(lr, param_grid = param,scoring ='roc_auc', cv=5)
gsearch.fit(X_train, y_train)

print('最佳参数：',gsearch.best_params_)
print('训练集的最佳分数：', gsearch.best_score_)
print('测试集的最佳分数：', gsearch.score(X_test, y_test))

D:\Anaconda2\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


('\xe6\x9c\x80\xe4\xbd\xb3\xe5\x8f\x82\xe6\x95\xb0\xef\xbc\x9a', {'penalty': 'l1', 'C': 0.1})
('\xe8\xae\xad\xe7\xbb\x83\xe9\x9b\x86\xe7\x9a\x84\xe6\x9c\x80\xe4\xbd\xb3\xe5\x88\x86\xe6\x95\xb0\xef\xbc\x9a', 0.7920261042203866)
('\xe6\xb5\x8b\xe8\xaf\x95\xe9\x9b\x86\xe7\x9a\x84\xe6\x9c\x80\xe4\xbd\xb3\xe5\x88\x86\xe6\x95\xb0\xef\xbc\x9a', 0.7794330902527828)


In [6]:
lr = LogisticRegression(C = 0.01, penalty = 'l2')
lr.fit(X_train, y_train)
model_metrics(lr, X_train, X_test, y_train, y_test)

[准确率]
训练集：0.8031
测试集：0.7884
[精准率]
训练集：0.7096
测试集：0.6667
[召回率]
训练集：0.3633
测试集：0.3175
[f1-score]
训练集：0.4806
测试集：0.4302
[auc值]
训练集：0.8147
测试集：0.7757


### SVM模型
    
    调参范围可设为'gamma':[0.001,0.01,0.1,1,10,100], 'C':[0.001,0.01,0.1,1,10,100]}。鉴于时间原因, 下面网格搜索时选用较小区间。

In [7]:
# 线性SVM
svm_linear = svm.SVC(kernel = 'linear', probability=True)
param = {'C':[0.01,0.1,1]}
gsearch = GridSearchCV(svm_linear, param_grid = param,scoring ='roc_auc', cv=5)
gsearch.fit(X_train, y_train)

print('最佳参数：',gsearch.best_params_)
print('训练集的最佳分数：', gsearch.best_score_)
print('测试集的最佳分数：', gsearch.score(X_test, y_test))

('\xe6\x9c\x80\xe4\xbd\xb3\xe5\x8f\x82\xe6\x95\xb0\xef\xbc\x9a', {'C': 0.01})
('\xe8\xae\xad\xe7\xbb\x83\xe9\x9b\x86\xe7\x9a\x84\xe6\x9c\x80\xe4\xbd\xb3\xe5\x88\x86\xe6\x95\xb0\xef\xbc\x9a', 0.7959313069304018)
('\xe6\xb5\x8b\xe8\xaf\x95\xe9\x9b\x86\xe7\x9a\x84\xe6\x9c\x80\xe4\xbd\xb3\xe5\x88\x86\xe6\x95\xb0\xef\xbc\x9a', 0.7806041542779046)


In [8]:
svm_linear = svm.SVC(C = 0.01, kernel = 'linear', probability=True)
svm_linear.fit(X_train, y_train)
model_metrics(svm_linear, X_train, X_test, y_train, y_test)

[准确率]
训练集：0.7998
测试集：0.7758
[精准率]
训练集：0.7897
测试集：0.6757
[召回率]
训练集：0.2746
测试集：0.2089
[f1-score]
训练集：0.4075
测试集：0.3191
[auc值]
训练集：0.8156
测试集：0.7806


In [9]:
# 多项式SVM
svm_poly = svm.SVC(kernel = 'poly', probability=True)
param = {'C':[0.01,0.1,1]}
gsearch = GridSearchCV(svm_poly, param_grid = param,scoring ='roc_auc', cv=5)
gsearch.fit(X_train, y_train)

print('最佳参数：',gsearch.best_params_)
print('训练集的最佳分数：', gsearch.best_score_)
print('测试集的最佳分数：', gsearch.score(X_test, y_test))

D:\Anaconda2\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


('\xe6\x9c\x80\xe4\xbd\xb3\xe5\x8f\x82\xe6\x95\xb0\xef\xbc\x9a', {'C': 0.01})
('\xe8\xae\xad\xe7\xbb\x83\xe9\x9b\x86\xe7\x9a\x84\xe6\x9c\x80\xe4\xbd\xb3\xe5\x88\x86\xe6\x95\xb0\xef\xbc\x9a', 0.7444664162105683)
('\xe6\xb5\x8b\xe8\xaf\x95\xe9\x9b\x86\xe7\x9a\x84\xe6\x9c\x80\xe4\xbd\xb3\xe5\x88\x86\xe6\x95\xb0\xef\xbc\x9a', 0.7307048292698194)


In [10]:
svm_poly =  svm.SVC(C = 0.01, kernel = 'poly', probability=True)
svm_poly.fit(X_train, y_train)
model_metrics(svm_poly, X_train, X_test, y_train, y_test)

[准确率]
训练集：0.7571
测试集：0.7505
[精准率]
训练集：1.0000
测试集：0.6364
[召回率]
训练集：0.0312
测试集：0.0195
[f1-score]
训练集：0.0605
测试集：0.0378
[auc值]
训练集：0.8658
测试集：0.7307


In [11]:
# 高斯SVM
svm_rbf = svm.SVC(probability=True)
param = {'gamma':[0.01,0.1,1,10], 
         'C':[0.01,0.1,1]}
gsearch = GridSearchCV(svm_poly, param_grid = param,scoring ='roc_auc', cv=5)
gsearch.fit(X_train, y_train)

print('最佳参数：',gsearch.best_params_)
print('训练集的最佳分数：', gsearch.best_score_)
print('测试集的最佳分数：', gsearch.score(X_test, y_test))

('\xe6\x9c\x80\xe4\xbd\xb3\xe5\x8f\x82\xe6\x95\xb0\xef\xbc\x9a', {'C': 0.01, 'gamma': 0.01})
('\xe8\xae\xad\xe7\xbb\x83\xe9\x9b\x86\xe7\x9a\x84\xe6\x9c\x80\xe4\xbd\xb3\xe5\x88\x86\xe6\x95\xb0\xef\xbc\x9a', 0.7466917074276515)
('\xe6\xb5\x8b\xe8\xaf\x95\xe9\x9b\x86\xe7\x9a\x84\xe6\x9c\x80\xe4\xbd\xb3\xe5\x88\x86\xe6\x95\xb0\xef\xbc\x9a', 0.7342441029493079)


In [12]:
svm_rbf =  svm.SVC(gamma = 0.01, C =0.01 , probability=True)
svm_rbf.fit(X_train, y_train)
model_metrics(svm_rbf, X_train, X_test, y_train, y_test)


[准确率]
训练集：0.7493
测试集：0.7484
[精准率]
训练集：0.0000
测试集：0.0000
[召回率]
训练集：0.0000
测试集：0.0000
[f1-score]
训练集：0.0000
测试集：0.0000
[auc值]
训练集：0.8510
测试集：0.7681


D:\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [13]:
# sigmoid - SVM
svm_sigmoid = svm.SVC(kernel = 'sigmoid',probability=True)
param = {'C':[0.01,0.1,1]}
gsearch = GridSearchCV(svm_sigmoid, param_grid = param,scoring ='roc_auc', cv=5)
gsearch.fit(X_train, y_train)

print('最佳参数：',gsearch.best_params_)
print('训练集的最佳分数：', gsearch.best_score_)
print('测试集的最佳分数：', gsearch.score(X_test, y_test))

('\xe6\x9c\x80\xe4\xbd\xb3\xe5\x8f\x82\xe6\x95\xb0\xef\xbc\x9a', {'C': 0.01})
('\xe8\xae\xad\xe7\xbb\x83\xe9\x9b\x86\xe7\x9a\x84\xe6\x9c\x80\xe4\xbd\xb3\xe5\x88\x86\xe6\x95\xb0\xef\xbc\x9a', 0.7754756970987209)
('\xe6\xb5\x8b\xe8\xaf\x95\xe9\x9b\x86\xe7\x9a\x84\xe6\x9c\x80\xe4\xbd\xb3\xe5\x88\x86\xe6\x95\xb0\xef\xbc\x9a', 0.7672503729669391)


In [14]:
svm_sigmoid =  svm.SVC(C = 0.01, kernel = 'sigmoid',probability=True)
svm_sigmoid.fit(X_train, y_train)
model_metrics(svm_sigmoid, X_train, X_test, y_train, y_test)

[准确率]
训练集：0.7493
测试集：0.7484
[精准率]
训练集：0.0000
测试集：0.0000
[召回率]
训练集：0.0000
测试集：0.0000
[f1-score]
训练集：0.0000
测试集：0.0000
[auc值]
训练集：0.7784
测试集：0.7673


### 决策树

1）首先对决策树最大深度max_depth和内部节点再划分所需最小样本数min_samples_split进行网格搜索。

In [15]:
param = {'max_depth':range(3,14,2), 'min_samples_split':range(100,801,200)}
gsearch = GridSearchCV(DecisionTreeClassifier(max_depth=8,min_samples_split=300,min_samples_leaf=20, max_features='sqrt',random_state =2333),
                       param_grid = param,scoring ='roc_auc', cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'max_depth': 13, 'min_samples_split': 100}, 0.7085909474219738)

2）对内部节点再划分所需最小样本数min_samples_split和叶子节点最少样本数min_samples_leaf一起调参。

In [16]:
param = {'min_samples_split':range(50,1000,100), 'min_samples_leaf':range(60,101,10)}
gsearch = GridSearchCV(DecisionTreeClassifier(max_depth=11,min_samples_split=100,min_samples_leaf=20, max_features='sqrt', random_state =2333),
                       param_grid = param,scoring ='roc_auc', cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'min_samples_leaf': 70, 'min_samples_split': 50}, 0.7034962764263947)

3）再对最大特征数max_features进行网格搜索

In [17]:
param = {'max_features':range(7,20,2)}
gsearch = GridSearchCV(DecisionTreeClassifier(max_depth=11,min_samples_split=550,min_samples_leaf=80, max_features='sqrt', random_state =2333),
                       param_grid = param,scoring ='roc_auc', cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'max_features': 19}, 0.712726329526747)

In [18]:
dt = DecisionTreeClassifier(max_depth=5,min_samples_split=50,min_samples_leaf=90,max_features=17, random_state =2018)
dt.fit(X_train, y_train)
model_metrics(dt, X_train, X_test, y_train, y_test)

[准确率]
训练集：0.7893
测试集：0.7666
[精准率]
训练集：0.6468
测试集：0.5644
[召回率]
训练集：0.3513
测试集：0.3175
[f1-score]
训练集：0.4553
测试集：0.4064
[auc值]
训练集：0.7777
测试集：0.7251


### XGBoost模型

1、max_depth = 5 :这个参数的取值最好在3-10之间。我选的起始值为5，但是你也可以选择其它的值。起始值在4-6之间都是不错的选择。
2、min_child_weight = 1:在这里选了一个比较小的值，因为这是一个极不平衡的分类问题。因此，某些叶子节点下的值会比较小。
3、gamma = 0: 起始值也可以选其它比较小的值，在0.1到0.2之间就可以。这个参数后继也是要调整的。
4、subsample, colsample_bytree = 0.8: 这个是最常见的初始值了。典型值的范围在0.5-0.9之间。

首先看一下默认参数的结果

In [19]:
import warnings
warnings.filterwarnings("ignore")

xgb0 = XGBClassifier()
xgb0.fit(X_train, y_train)

model_metrics(xgb0, X_train, X_test, y_train, y_test)

[准确率]
训练集：0.8503
测试集：0.7821
[精准率]
训练集：0.8636
测试集：0.6176
[召回率]
训练集：0.4784
测试集：0.3510
[f1-score]
训练集：0.6157
测试集：0.4476
[auc值]
训练集：0.9183
测试集：0.7695


1) 首先从步长(learning rate)和迭代次数(n_estimators)入手。
开始选择一个较小的步长来网格搜索最好的迭代次数。这里，我们将步长初始值设置为0.1, 对于迭代次数进行网格搜索。

In [20]:
param_test = {'n_estimators':range(20,200,20)}
gsearch = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=140, max_depth=5, 
                                                  min_child_weight=1, gamma=0, subsample=0.8, 
                                                  colsample_bytree=0.8, objective= 'binary:logistic', 
                                                  nthread=4,scale_pos_weight=1, seed=27), 
                        param_grid = param_test, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'n_estimators': 40}, 0.7974625809665962)

2) max_depth 和 min_child_weight 参数调优

In [21]:
param_test = {'max_depth':range(3,10,2), 'min_child_weight':range(1,12,2)}

gsearch = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=40, max_depth=5, 
                                                  min_child_weight=1, gamma=0, subsample=0.8, 
                                                  colsample_bytree=0.8, objective= 'binary:logistic', 
                                                  nthread=4,scale_pos_weight=1, seed=27), 
                        param_grid = param_test, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'max_depth': 5, 'min_child_weight': 9}, 0.7996564194995348)

可以看出理想的max_depth值为5，理想的min_child_weight值为3。在这个值附近我们可以再进一步调整，来找出理想值。

In [22]:
param_test = {'max_depth':[3,4,5], 'min_child_weight':[3,4,5]}

gsearch = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=40, max_depth=3, 
                                                  min_child_weight=1, gamma=0, subsample=0.8, 
                                                  colsample_bytree=0.8, objective= 'binary:logistic', 
                                                  nthread=4,scale_pos_weight=1, seed=27), 
                        param_grid = param_test, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'max_depth': 4, 'min_child_weight': 3}, 0.8046473698741362)

3) gamma参数调优

In [23]:
param_test = {'gamma':[i/10 for i in range(1,6)]}

gsearch = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=40, max_depth=3, 
                                                  min_child_weight=3, gamma=0, subsample=0.8, 
                                                  colsample_bytree=0.8, objective= 'binary:logistic', 
                                                  nthread=4,scale_pos_weight=1, seed=27), 
                        param_grid = param_test, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'gamma': 0}, 0.7977344545174377)

4）调整subsample 和 colsample_bytree 参数

In [24]:
param_test = {'subsample':[i/10 for i in range(5,10)], 'colsample_bytree':[i/10 for i in range(5,10)]}

gsearch = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=40, max_depth=3, 
                                                  min_child_weight=3, gamma=0.1, subsample=0.8, 
                                                  colsample_bytree=0.8, objective= 'binary:logistic', 
                                                  nthread=4,scale_pos_weight=1, seed=27), 
                        param_grid = param_test, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'colsample_bytree': 0, 'subsample': 0}, 0.5)

从这里可以看出来，subsample理想取值0.8, colsample_bytree理想取值都是0.6。现在，我们以0.05为步长，在这个值附近尝试取值。

In [25]:
param_test = { 'subsample':[i/100 for i in range(85,101,5)], 'colsample_bytree':[i/100 for i in range(85,101,5)]}

gsearch = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=40, max_depth=3, 
                                                  min_child_weight=3, gamma=0.1, subsample=0.9, 
                                                  colsample_bytree=0.5, objective= 'binary:logistic', 
                                                  nthread=4,scale_pos_weight=1, seed=27), 
                        param_grid = param_test, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'colsample_bytree': 1, 'subsample': 1}, 0.799949770230203)

5）正则化参数调优
        
        'reg_lambda': [0.2, 0.4, 0.6, 0.8, 1]

In [26]:
param_test = {'reg_alpha':[1e-5, 1e-2, 0.1, 0, 1, 100]}

gsearch = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=40, max_depth=3, 
                                                  min_child_weight=3, gamma=0.1, subsample=0.9, 
                                                  colsample_bytree=0.85, objective= 'binary:logistic', 
                                                  nthread=4,scale_pos_weight=1, seed=27), 
                        param_grid = param_test, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'reg_alpha': 1}, 0.8029492568543652)

6）继续从1）循环调整, 此处省略步骤。

7）回到第1）步，降低学习速率, 调整迭代次数

In [27]:
param_test = {'n_estimators':range(20,200,20)}

gsearch = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=40, max_depth=3, 
                                                  min_child_weight=3, gamma=0.1, subsample=0.9, 
                                                  colsample_bytree=0.85, reg_alpha=1e-05, objective= 'binary:logistic', 
                                                  nthread=4,scale_pos_weight=1, seed=27), 
                        param_grid = param_test, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'n_estimators': 40}, 0.8027894462281872)

与1）结果相比, 较好。所以, 选用此处学习速率较小时的结果。继续从1）循环调整, 此处不再调整。

In [28]:
xgb = XGBClassifier(learning_rate =0.1, n_estimators=60, max_depth=3, min_child_weight=3, 
                    gamma=0.1, subsample=0.9, colsample_bytree=0.85, reg_alpha=1e-05, 
                    objective= 'binary:logistic', nthread=4,scale_pos_weight=1, seed=27)
xgb.fit(X_train, y_train)
model_metrics(xgb, X_train, X_test, y_train, y_test)

[准确率]
训练集：0.8326
测试集：0.7912
[精准率]
训练集：0.8184
测试集：0.6631
[召回率]
训练集：0.4269
测试集：0.3454
[f1-score]
训练集：0.5611
测试集：0.4542
[auc值]
训练集：0.8830
测试集：0.7764


### LightGBM模型

In [29]:
# 首先看一下默认参数的结果
lgb0 = LGBMClassifier()
lgb0.fit(X_train, y_train)

model_metrics(lgb0, X_train, X_test, y_train, y_test)

[准确率]
训练集：0.9946
测试集：0.7722
[精准率]
训练集：1.0000
测试集：0.5810
[召回率]
训练集：0.9784
测试集：0.3398
[f1-score]
训练集：0.9891
测试集：0.4288
[auc值]
训练集：1.0000
测试集：0.7605


In [30]:
param_test = {'n_estimators':range(20,200,20)}
gsearch = GridSearchCV(estimator = LGBMClassifier(learning_rate =0.1, n_estimators=60, max_depth=3, 
                                                  min_child_weight=11, gamma=0.1, subsample=0.5, 
                                                  colsample_bytree=0.8, reg_alpha = 1e-5,
                                                  nthread=4,scale_pos_weight=1, seed=27), 
                        param_grid = param_test, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 

gsearch.best_params_, gsearch.best_score_

({'n_estimators': 40}, 0.8026840319438502)

In [31]:
lgb = LGBMClassifier(learning_rate =0.1, n_estimators=40, max_depth=3, min_child_weight=11, 
                    gamma=0.1, subsample=0.5, colsample_bytree=0.8, reg_alpha=1e-5, 
                    nthread=4,scale_pos_weight=1, seed=27)
lgb.fit(X_train, y_train)
model_metrics(lgb, X_train, X_test, y_train, y_test)

[准确率]
训练集：0.8275
测试集：0.7905
[精准率]
训练集：0.8171
测试集：0.6648
[召回率]
训练集：0.4017
测试集：0.3370
[f1-score]
训练集：0.5386
测试集：0.4473
[auc值]
训练集：0.8634
测试集：0.7776


|模型|参数|准确率|auc值|
|:---|:---|:---|:---|
|LR|LogisticRegression(C = 0.01, penalty = 'l2')|训练集： 0.8013 测试集： 0.7877|训练集： 0.8136 测试集： 0.7786|
|svm_linear|svm.SVC(C = 0.01, kernel = 'linear', probability=True)|训练集： 0.8001 测试集： 0.7765|训练集： 0.8155 测试集： 0.7810|
|svm_poly|svm.SVC(C = 0.01, kernel = 'poly', probability=True)|训练集： 0.7538 测试集： 0.7547| 训练集： 0.8690 测试集： 0.7260|
|svm_rbf|svm.SVC(gamma = 0.01, C =0.01 , probability=True)|训练集： 0.7493 测试集： 0.7484|训练集： 0.8454 测试集： 0.7687|
|svm_sigmoid|svm.SVC(C = 0.01, kernel = 'sigmoid',probability=True)|训练集： 0.7493 测试集： 0.7484|训练集： 0.7746 测试集： 0.7619|
|决策树|DecisionTreeClassifier(max_depth=5,min_samples_split=300,min_samples_leaf=90,max_features=17)|训练集： 0.7650 测试集： 0.7449|训练集： 0.7509 测试集： 0.6879|
|XGBoost|XGBClassifier(learning_rate =0.01, n_estimators=180, max_depth=3, min_child_weight=5,gamma=0.4, subsample=0.5, colsample_bytree=0.9, reg_alpha=1, objective= 'binary:logistic',nthread=4,scale_pos_weight=1, seed=27)|训练集： 0.8142 测试集： 0.7835|训练集： 0.8318 测试集： 0.7680|
|LightGBM|LGBMClassifier(learning_rate =0.1, n_estimators=40, max_depth=3, min_child_weight=11,gamma=0.1, subsample=0.5, colsample_bytree=0.8, reg_alpha=1e-5, nthread=4,scale_pos_weight=1, seed=27)|训练集： 0.8308 测试集： 0.7926|训练集： 0.8791 测试集： 0.7756|